In [27]:
import pandas as pd
import pyodbc 

In [28]:
df = pd.read_csv('Repo_in2023.csv')

In [29]:
df['created_at'] = pd.to_datetime(df['created_at'])
# Extract the date part from the datetime and convert back to string format :
df['created_at'] = df['created_at'].dt.strftime('%Y-%m-%d')

In [30]:
df['contributors']  = df['contributors'].apply(eval)
df['contributions_count']  = df['contributions_count'].apply(eval)

In [31]:
df.head(1)

,name,url,description,stars,created_at,language,forks,open_issues,owner,contributors_count,contributors,contributions_count
0,learn-javascript,https://github.com/sumn2u/learn-javascript,A book that teaches JavaScript,635,2023-01-01,HTML,32,1,sumn2u,6,"[gnwx, sumn2u, Anuj-Khadka, sahilxkhadka, hema...","[83, 64, 2, 2, 1, 1]"


#### Create Conection avec la base de donnee :

In [32]:
# Créer un connexion dans le database :

driver = 'SQL Server'
server = 'LAPTOP-6D8J0VI4\SQLEXPRESS'
database = 'github_DB'

conn = pyodbc.connect(f'''
                        DRIVER={driver};
                        SERVER={server};
                        DATABASE={database};
                        Trusted_Connection=yes
                       ''')
cursor = conn.cursor()
conn

In [ ]:
cursor.execute('''
               
CREATE TABLE [Languages] (
  [Id] INT IDENTITY(0,1),
  [name] VARCHAR(255),
  PRIMARY KEY ([Id])
);

CREATE TABLE [Owner] (
  [Id] INT IDENTITY(0,1),
  [name] VARCHAR(255),
  PRIMARY KEY ([Id])
);

CREATE TABLE [Repositorys] (
  [Id] INT IDENTITY(0,1),
  [Url] VARCHAR(255),
  [description] VARCHAR(max),
  [forks] INT,
  [created_at] DATE,
  [open_issues] INT,
  [contributors_count] INT,
  [language_id] INT,
  [owner_id] INT,
  PRIMARY KEY ([Id]),
  CONSTRAINT [FK_Repositorys_language_id]
    FOREIGN KEY ([language_id])
    REFERENCES [Languages]([Id]),
  CONSTRAINT [FK_Repositorys_owner_id]
    FOREIGN KEY ([owner_id])
    REFERENCES [Owner]([Id])
);


CREATE TABLE [Contributors] (
  [Id] INT IDENTITY(0,1),
  [name] VARCHAR(255),
  [number_commit] INT
  PRIMARY KEY ([Id])
);

CREATE TABLE [Contributor_bridge] (
  [Id_repo] INT,
  [Id_contributor] INT
);

-- Add Constraints

ALTER TABLE [Contributor_bridge]
ADD CONSTRAINT [FK_Contributor_bridge_Id_repo]
  FOREIGN KEY ([Id_repo])
  REFERENCES [Repositorys]([Id]);

ALTER TABLE [Contributor_bridge]
ADD CONSTRAINT [FK_Contributor_bridge_Id_contributor]
  FOREIGN KEY ([Id_contributor])
  REFERENCES [Contributors]([Id]);
        ''')

cursor.commit()

In [7]:
tables = cursor.tables(tableType='TABLE')

for table_info in tables:
    table_name = table_info.table_name
    print(table_name)

Contributor_bridge
Contributors
Languages
Owner
Repositorys
trace_xe_action_map
trace_xe_event_map


#### Insert Into language Table :

In [8]:
uique_Language  = list(df['language'].unique())

for languge in uique_Language :
    cursor.execute('INSERT INTO Languages(name) VALUES (?)',languge)

cursor.commit()

In [12]:
len(uique_Language)

146

#### Insert Into owner Table :

In [9]:
owners  = list(df['owner'].unique())

for owner in owners :
    cursor.execute('INSERT INTO Owner(name) VALUES (?)',owner)

cursor.commit()

#### Insert Into Contributor Table :

In [10]:
list_contributions_count = list(df['contributions_count'])
list_contributions = list(df['contributors'])

list_contributions = [element for sublist in list_contributions for element in sublist]
list_contributions_count = [element for sublist in list_contributions_count for element in sublist]

# Create a dictionary from the lists :
data = {'Contributor': list_contributions, 'Contribution_Count': list_contributions_count}

# Create a DataFrame from the dictionary :
Conterbuter_df = pd.DataFrame(data)
Conterbuter_df = Conterbuter_df.drop_duplicates()
Conterbuter_df.duplicated().sum()


0

In [11]:
for index, row in Conterbuter_df.iterrows() :
    cursor.execute('INSERT INTO Contributors(name,number_commit) VALUES (?,?)',row['Contributor'],row['Contribution_Count'])

cursor.commit()

#### Insert Into Repos Table :

In [ ]:
for index, row in df.iterrows() :

    cursor.execute(f"SELECT id FROM Owner WHERE name = '{row['owner']}'")
    owner_id = cursor.fetchone()[0]

    cursor.execute(f"SELECT id FROM Languages WHERE name = '{row['language']}'")
    language_id = cursor.fetchone()[0]
    
    cursor.execute('INSERT INTO Repositorys(Url,description,forks,created_at,open_issues,contributors_count,language_id,owner_id) VALUES (?,?,?,?,?,?,?,?)',
    row['url'],
    str(row['description']),
    row['forks'],
    row['created_at'],
    row['open_issues'],
    row['contributors_count'],
    language_id, 
    owner_id
    )

cursor.commit()

#### insert into bridg :

In [16]:
df.head(1)

,name,url,description,stars,created_at,language,forks,open_issues,owner,contributors_count,contributors,contributions_count
0,learn-javascript,https://github.com/sumn2u/learn-javascript,A book that teaches JavaScript,635,2023-01-01,HTML,32,1,sumn2u,6,"[gnwx, sumn2u, Anuj-Khadka, sahilxkhadka, hema...","[83, 64, 2, 2, 1, 1]"


In [ ]:
for index, row in df.iterrows() :
    url = row['url']
    contributors = row['contributors']

    cursor.execute(f"SELECT id FROM Repositorys WHERE Url = '{url}'")
    Repositorys_id = cursor.fetchone()[0]

    for i in range(0,len(contributors)) :
        cursor.execute(f"SELECT id FROM Contributors WHERE name = '{contributors[i]}'")
        Contributors_id = cursor.fetchone()[0]
        print(Repositorys_id,'=>',Contributors_id)
        cursor.execute('INSERT INTO Contributor_bridge(Id_repo,Id_contributor) VALUES (?,?)',Repositorys_id,Contributors_id)

cursor.commit()

In [39]:
conn.close()